In [ ]:
# import libraries
import nltk
import random
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import string
import re
from fuzzywuzzy import process

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
import sqlite3

In [ ]:
# define DB class that interacts with the database

class Db:
    def __init__(self):
        self.conn_string = 'data/university_chatbot.db'
        self.init_db()
    
    def is_already_initialized(self):
        return self.get_one("SELECT name FROM sqlite_master WHERE type='table' AND name='students'")

    def init_db(self):

        if self.is_already_initialized():
            return
        
        # Create tables
        # Students Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS students (
                matriculation_number INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                surname TEXT NOT NULL,
                address TEXT
            )
            ''')

        # Courses Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS courses (
                course_id INTEGER PRIMARY KEY,
                course_name TEXT NOT NULL,
                instructor TEXT NOT NULL
            )
            ''')

        # Registrations Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS registrations (
                registration_id INTEGER PRIMARY KEY,
                matriculation_number INTEGER NOT NULL,
                course_id INTEGER NOT NULL,
                FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
                FOREIGN KEY (course_id) REFERENCES courses (course_id)
            )
            ''')

        # Exam Results Table
        self.execute_mutation('''
            CREATE TABLE IF NOT EXISTS exam_results (
                result_id INTEGER PRIMARY KEY,
                matriculation_number INTEGER NOT NULL,
                course_id INTEGER NOT NULL,
                grade TEXT NOT NULL,
                FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
                FOREIGN KEY (course_id) REFERENCES courses (course_id)
            )
            ''')
        self.fill_db_with_sample_data()

    def fill_db_with_sample_data(self):
        # Sample data for students
        students = [
            (201001, 'Alice', 'Smith', 'Jakobstraße 123'),
            (201002, 'Bob', 'Johnson', 'Schlägelstraße 456'),
            (201003, 'Carol', 'Williams', 'Borsigplatz 789'),
            (201004, 'David', 'Brown', 'Kamener Straße 101'),
            (201005, 'Eve', 'Davis', 'Priorstraße 202'),
            (201006, 'Frank', 'Miller', 'Burgholzstraße 303'),
            (201007, 'Grace', 'Wilson', 'Clausthaler Straße 404'),
            (201008, 'Henry', 'Moore', 'Nordstraße 505'),
            (201009, 'Ivy', 'Taylor', 'Am Waldfried 606'),
            (201010, 'Jack', 'Anderson', 'Westfalenhüttenallee 707')
        ]
        # Sample data for courses
        courses = [
            (101, 'Introduction to Computer Science', 'Dr. John Doe'),
            (102, 'Advanced Mathematics', 'Dr. Jane Smith'),
            (103, 'Physics for Engineers', 'Dr. Emily Johnson'),
            (104, 'Data Structures and Algorithms', 'Dr. Alan Turing'),
            (105, 'Database Systems', 'Dr. Edgar Codd'),
            (106, 'Artificial Intelligence', 'Dr. Ada Lovelace'),
            (107, 'Machine Learning', 'Dr. Geoffrey Hinton'),
            (108, 'Software Engineering', 'Dr. Grace Hopper'),
            (109, 'Web Development', 'Dr. Tim Berners-Lee'),
            (110, 'Operating Systems', 'Dr. Linus Torvalds'),
            (111, 'Computer Networks', 'Dr. Vint Cerf')
        ]

        # Insert students into the database
        self.execute_mass_mutation('INSERT INTO students VALUES (?,?,?,?)', students)

        # Insert courses into the database
        self.execute_mass_mutation('INSERT INTO courses VALUES (?,?,?)', courses)

        # Generate registrations (6 for each student)
        registrations = []
        for student in students:
            registered_courses = random.sample(courses, 6)  # Randomly pick 6 courses for each student
            for course in registered_courses:
                registrations.append((None, student[0], course[0]))

        # Insert registrations into the database
        self.execute_mass_mutation('INSERT INTO registrations (registration_id, matriculation_number, course_id) VALUES (?,?,?)', registrations)

        # Generate exam results (4 for each student)
        exam_results = []
        for student in students:
            course_ids = random.sample(registrations,4)  # Randomly pick 4 course from registrations
            for course_id in course_ids:
                grade = random.choice(['1.0','1.3','1.5','1.7', '2.0', '2.3','2.5','2.7', '3.0','3.3','3.5','3.7', '4.0', '5.0'])  # Randomly assign a grade
                exam_results.append((None, student[0], course_id[2], grade))
        # Insert exam results into the database
        self.execute_mass_mutation('INSERT INTO exam_results (result_id, matriculation_number, course_id, grade) VALUES (?,?,?,?)', exam_results)
            

    def get_one(self, query, args=()):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.execute(query, args)
        result = cursor.fetchone()
        conn.close()
        return result
    
    def get_many(self, query):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()
        return result
    
    def execute_mutation(self, query, args = ()):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.execute(query, args)
        conn.commit()
        conn.close()

    def execute_mass_mutation(self, query, params = []):
        conn = sqlite3.connect(self.conn_string)
        cursor = conn.cursor()
        cursor.executemany(query, params)
        conn.commit()
        conn.close()   


In [ ]:
class AbortError(Exception):
    "The user wants to abort the conversation"
    pass

class ChatBot:
    def __init__(self):
        self.db = Db()
        self.prompts = []
        self.answers = []
        self.intent = None
        self.matric_number_pattern = r'\b\d{6}\b'
        self.filtered_surnames = []
        self.street_filtered_combinations = []
        self.city_filtered_combinations = []
        self.intent_keywords = {
            'register_exam': [
                'register', 'enroll', 'signup', 'apply', 'sign-up', 'join', 'participate', 
                'exam', 'examination', 'registration', 'enrollment', 'enrolling', 'registering', 
                'application', 'applying', 'signing-up', 'joining', 'participation'
            ],
            'deregister_exam': [
                'deregister', 'withdraw', 'cancel', 'unenroll', 'opt-out', 'drop', 'leave', 
                'removal', 'exit', 'unregister', 'withdrawing', 'canceling', 'unenrolling', 
                'dropping', 'leaving', 'exiting', 'opting-out'
            ],
            'query_exam_status': [
                'query', 'inquire', 'ask', 'check', 'verify', 'confirm', 'status', 'update', 
                'detail', 'information', 'enrollment-status', 'registration-status', 'exam-status', 
                'current-status', 'status-check', 'verification', 'confirmation', 'enrolled', 
                'registered', 'inquiring', 'asking', 'checking', 'verifying', 'confirming'
            ],
            'query_exam_grade': [
                'grade', 'score', 'result', 'mark', 'performance', 'outcome', 'rating', 
                'evaluation', 'assessment', 'exam-result', 'grade-check', 'score-check', 
                'result-inquiry', 'performance-evaluation', 'rating-inquiry', 'exam-grade', 
                'exam-score', 'exam-mark', 'exam-performance'
            ],
            'change_address': [
                'change', 'update', 'modify', 'new', 'relocate', 'move', 'alter', 'edit', 
                'address', 'location', 'residence', 'changing', 'updating', 'modifying', 
                'relocating', 'moving', 'altering', 'editing'
            ],
            'change_surname': [
                'change', 'update', 'modify', 'alter', 'edit', 'surname', 'last-name', 
                'family-name', 'name-change', 'name-modification', 'name-update', 'name-alteration', 
                'changing', 'updating', 'modifying', 'altering', 'editing', 'marriage', 'divorce'
            ],
            'suggest_course': [
                'suggest', 'recommend', 'propose', 'advise', 'course', 'recommendation', 
                'suggestion', 'proposal', 'advice', 'course-suggestion', 'course-recommendation', 
                'course-proposal', 'course-advice', 'suggesting', 'recommending', 'proposing'
            ],
            'test': [
                'test', 'trial', 'experiment', 'quiz', 'assessment', 'evaluation', 'exam', 
                'testing', 'trial-run', 'experimenting', 'quizzing', 'assessing', 'evaluating'
            ],
            'abort': [
                'abort', 'stop', 'end', 'cancel', 'terminate', 'halt', 'quit', 'cease', 
                'exit', 'interrupt', 'suspend', 'discontinue', 'stopping', 'ending', 'canceling', 
                'terminating', 'halting', 'quitting', 'ceasing', 'exiting', 'interrupting', 'suspending'
            ]
        }
    
    def chat(self, prompt):
        print("Bot: " + prompt)
        self.prompts.append(prompt)
        answer = input()
        print("You: "+ answer)
        self.answers.append(answer)
        self.check_for_abort(answer)
        #self.process_input(answer)
        return answer

    def check_for_abort(self, answer):
       words = self.process_input(answer)
       abort_keywords = self.intent_keywords['abort']
       if any(keyword in words for keyword in abort_keywords):
           raise AbortError()
    
    #Try to Identify Intents and repeat until a single Intent is found
    def start_conversation(self):
        try:
            response = self.chat("Lets get started")
            while self.intent is None:

                #check response for Intents
                words = self.process_input(response)
                intents = self.identify_intent(words)
                if intents[0] == "unknown_intent":
                    self.intent = intents[0]
                    self.act()
                    #If we come back from the action we need a new response
                    response = self.chat("What shall we do now?")
                else:
                    intents = self.prioritize_intents(words)
                    #Just for Debugging
                    #print("I have identified the following intent: ", intents[0])
                    self.intent = intents[0]
                    self.act()
                    print("We have processed your request!")
                    self.intent = None
                    response = self.chat("What shall we do now?")
        except AbortError:
            self.abort()
            self.start_conversation()

    def get_wordnet_pos(self, word):
        #"""Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    def combine_keywords(self):
        all_keywords = []
        for keywords in self.intent_keywords.values():
            all_keywords.extend(keywords)
        return all_keywords    
            
    def process_input(self, user_input = None):
        if user_input is None:
            user_input = self.answers[-1]

        tokens = word_tokenize(user_input.lower())
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]

       # Fuzzy Matching before Lemmatization
        all_keywords = self.combine_keywords()
        corrected_words = []
        for word in words:
            match = process.extractOne(word, all_keywords, score_cutoff=80)
            if match:
                best_match, score = match
                corrected_words.append(best_match)
            else:
                corrected_words.append(word)

        print("Debug - Corrected Words:", corrected_words)  # Debug print
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        lemmatized_words = [lemmatizer.lemmatize(w, self.get_wordnet_pos(w)) for w in corrected_words]

        print("Debug - Lemmatized Words:", lemmatized_words)  # Debug print
        return lemmatized_words
    
    def identify_intent(self, processed_input):
        matched_intents = []
        for intent,keywords in self.intent_keywords.items():
            if any(keyword in processed_input for keyword in keywords):
                matched_intents.append(intent)
        
        if not matched_intents:
            return ['unknown_intent']
        
        return matched_intents

    def prioritize_intents(self, words):
        # Initialize a dictionary to count occurrences for each intent
        intent_scores = {intent: 0 for intent in self.intent_keywords.keys()}

        for word in words:
            for intent, keywords in self.intent_keywords.items():
                for keyword in keywords:
                    if word == keyword:
                        intent_scores[intent] += 1
        sorted_intents = sorted(intent_scores, key=intent_scores.get, reverse=True)

        # Remove intents without hit
        prioritized_intents = [intent for intent in sorted_intents if intent_scores[intent] > 0]

        return prioritized_intents
    
    def act(self):
        if self.intent == 'unknown_intent':
            print("Sorry, I didn't understand.")
            self.intent = None
        elif self.intent == 'change_address':
            self.change_address()
        elif self.intent == 'change_surname':
            self.change_surname()        
        elif self.intent == 'register_exam':
            self.register_exam(),
        elif self.intent == 'deregister_exam':
            self.deregister_exam(),
        elif self.intent == 'query_exam_status':
            self.query_exam_status()
        elif self.intent == 'query_exam_grade':
            self.query_exam_grade()
        elif self.intent == 'suggest_course': 
            self.suggest_course()
        elif self.intent == 'abort':
            self.abort()
        elif self.intent == 'test':
            self.test()

        # Add more intents and associated actions here
    
    def has_postcode(user_input):
        postcode_pattern = re.compile(r'\b\d{5}\b')
        return bool(re.search(postcode_pattern, user_input))
    
    def identify_yes_no_answer(self, input):
        # check if input is string
        if isinstance(input, str):
            input = input.lower()
        intent_keywords = {
            'yes': ['true', 'yes','yeah', 'yep'],
            'no': ['wrong', 'false', 'no', 'not', 'nope', 'nah'],
            # Add more intents and associated keywords here
        }

        for intent, keywords in intent_keywords.items():
            if any(keyword in input for keyword in keywords):
                    return intent

        return 'unknown_intent'
    
    #Iterate backwards through answers to find matriculation number
    def find_matriculation_number(self, answers):
        for answer in reversed(answers):
            match = re.search(self.matric_number_pattern, answer.strip())
            if match:
         #       confirm = self.chat(f"Is your matriculation {match.group()}")
        #        processed_input = self.process_input(confirm)
        #        if self.identify_yes_no_answer(processed_input) == 'yes':
                return match.group()
        return None
        # Method to find course ID from user answers
    def find_course_id(self, answers):
        # Query the database for course names and IDs
        courses = self.db.get_many('SELECT course_id, course_name FROM courses')
        course_dict = {str(id): name.lower() for id, name in courses}

        for answer in reversed(answers):
            # Check for course ID (as a string) or course name in the answer
            for course_id_str, course_name in course_dict.items():
                if course_id_str in answer or course_name in answer.lower():
                    return int(course_id_str)  # Convert back to int before returning
        return None
    
    def change_surname(self):

        matriculation_number = self.find_matriculation_number(self.answers)
        while matriculation_number is None:
            self.chat("Please enter your matriculation number: ")
            matriculation_number = self.find_matriculation_number(self.answers)
            
        
        # Check if student exists
        student=  self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        
        if student is None:
            print("Sorry, you are not registered as a student.")
        else:
            # Ask for new surname
            surname = ''

            def validate_surname_input(new_surname_input, is_final_answer = False):
         
                tokens = word_tokenize(new_surname_input)
                tagged = nltk.pos_tag(tokens)
                temp_surname_filtered_combinations = [word[0] for word in tagged if word[1] in ['NNP', 'NN'] and word[0].isalpha() and word[0].lower() != 'surname' and word[0][0].isupper()]
                if len(temp_surname_filtered_combinations) > 0:
                    self.filtered_surnames = temp_surname_filtered_combinations
                if is_final_answer:
                    while_loop_counter = 0


                    different_surname_given_in_different_sentences = False

                    for surname in self.filtered_surnames:
                        if surname != self.filtered_surnames[0]:
                            different_surname_given_in_different_sentences = True
                            break

                    while len(self.filtered_surnames) == 0 or different_surname_given_in_different_sentences:
                        prompt = "Please make sure you have entered your new surname correctly."
                        if while_loop_counter == 0:
                            prompt = "Can you give me your new surname?"
                        new_surname_input = self.chat(prompt)
                        tokens = word_tokenize(new_surname_input)
                        tagged = nltk.pos_tag(tokens)
                        temp_surname_filtered_combinations = [word[0] for word in tagged if word[1] in ['NNP', 'NN'] and word[0].isalpha() and word[0].lower() != 'surname']
                        if len(temp_surname_filtered_combinations) > 0:
                            self.filtered_surnames = temp_surname_filtered_combinations
                            for surname in self.filtered_surnames:
                                if surname != self.filtered_surnames[0]:
                                    different_surname_given_in_different_sentences = True
                                    break
                        while_loop_counter += 1
                    surname = self.filtered_surnames[0]
                    return surname
            
            for index, answer in enumerate(reversed(self.answers)):
                surname = validate_surname_input(answer, index == len(self.answers) - 1)
                if surname is not None:
                    break
            

            answer = self.chat("Let me summarize once again: Your new surname is " + surname + "? ")
            processed_answer = self.identify_yes_no_answer(answer)
            
            while processed_answer == 'unknown_intent':
                answer = self.chat("Sorry, I didn't understand. Please answer with yes or no. ")
                processed_answer = self.identify_yes_no_answer(answer)
            
            while processed_answer == 'no':
                self.filtered_surnames = []
                new_surname_input = self.chat("Please enter your correct new surname: ")
                surname = validate_surname_input(new_surname_input, True)
                answer = self.chat("Let me summarize once again: Your new surname is " + surname + "? ")
                processed_answer = self.identify_yes_no_answer(answer)
            
            self.db.execute_mutation('UPDATE students SET surname=? WHERE matriculation_number=?', (surname, matriculation_number))

            print("Your surname has been updated.")
            # Update student's surname
    def query_exam_status(self):
        matriculation_number = self.find_matriculation_number(self.answers)
        course_id = self.find_course_id(self.answers)
        isMissingArguments = matriculation_number is None or course_id is None
        while isMissingArguments:
            if course_id is None and matriculation_number is None:
                self.chat("I could not identify your Course or your Matriculation number. Please enter the Matriculation number and the course name or course ID")
                matriculation_number = self.find_matriculation_number(self.answers)
                course_id = self.find_course_id(self.answers)
            elif course_id is None:
                self.chat("I could not identify your Course. Please enter the course name or course ID")
                course_id = self.find_course_id(self.answers)
            elif matriculation_number is None:
                self.chat("I could not identify your Matriculation number. Please enter the Matriculation number")
                matriculation_number = self.find_matriculation_number(self.answers)
            isMissingArguments = matriculation_number is None or course_id is None

        course_name = self.db.get_one('SELECT course_name FROM courses WHERE course_id = ?', (str(course_id),))
        matriculation_number = int(matriculation_number)

        confirmation = self.chat(f"Is it correct that you want to query the status of your exam {course_name}")
        processed_input = self.process_input(confirmation)
        if self.identify_yes_no_answer(processed_input) == 'yes':
            registration = self.db.get_many(f'SELECT * FROM registrations WHERE matriculation_number={matriculation_number} AND course_id={course_id}')
            if registration:
                print(f"You are registered for Course {course_id}!")
            else:
                print(f"No registration for Course {course_id} found!")

    def query_exam_grade(self):
        # Find matriculation number from stored answers or ask the user
        matriculation_number = self.find_matriculation_number(self.answers)
        course_id = self.find_course_id(self.answers)
        isMissingArguments = matriculation_number is None or course_id is None
        isEnlistedCourseShown = False
        while isMissingArguments:
            if course_id is None and matriculation_number is None:
                self.chat("I could not identify your Course or your Matriculation number. Please enter the Matriculation number and the course name or course ID")
                matriculation_number = self.find_matriculation_number(self.answers)
                course_id = self.find_course_id(self.answers)
            elif course_id is None:
                if isEnlistedCourseShown:
                    self.chat("I could not identify your Course. Please enter the course name or course ID")
                    course_id = self.find_course_id(self.answers)
                else:
                    query = f'''SELECT e.course_id, c.course_name 
                        FROM exam_results e 
                        JOIN courses c ON e.course_id = c.course_id 
                        WHERE e.matriculation_number = {matriculation_number}'''
                    completed_courses = self.db.get_many(query)
                    course_list = ', '.join([f"{course[0]} - {course[1]}" for course in completed_courses])
                    isEnlistedCourseShown = True
                    if len(completed_courses) == 1:
                        course_id = completed_courses[0][0]
                    elif len(completed_courses) == 0:
                        self.chat("You have no grade available yet")
                        return
                    else:
                        self.chat(f"Please choose a course to query the grade from the following: {course_list}")
                        course_id = self.find_course_id(self.answers)

            elif matriculation_number is None:
                self.chat("I could not identify your Matriculation number. Please enter the Matriculation number")
                matriculation_number = self.find_matriculation_number(self.answers)
            isMissingArguments = matriculation_number is None or course_id is None

        matriculation_number = int(matriculation_number)
        
        course_name = self.db.get_one('SELECT course_name FROM courses WHERE course_id = ?', (str(course_id),))
        confirmation = self.chat(f"Is it correct that you want to query the grade of your exam {course_name}")
        processed_input = self.process_input(confirmation)
        if self.identify_yes_no_answer(processed_input) == 'yes':
            # Query the database for the exam grade
            result = self.db.get_many(f'SELECT grade FROM exam_results WHERE matriculation_number={matriculation_number} AND course_id={course_id}')

            # Handle the response based on the query result
            if result:
                print(f"Your grade for Course {course_id} is {result[0][0]}")
            else:
                print("No grade available or Examination not yet passed")
            
# In case the user changes his Mind abort resets the state and 
    def abort(self):
         self.intent = None
         self.answers = []
         self.prompts = []
         self.filtered_surnames = []
         self.street_filtered_combinations = []
         self.city_filtered_combinations = []
        
# Test purpose
    def test(self):
        print("Test Test")
        print("Test Test")
        print("Test Test")

    def get_course_names_from_db(self):
        courses_data = self.db.get_many("SELECT course_name FROM courses")
        courses = [course[0] for course in courses_data]
        return courses

    def extract_course_name(self, texts):
        course_names = self.get_course_names_from_db()
        latest_course_name = None
        relevant_texts = []
        for text in reversed(texts):
            if text == "--START REGISTRATION--" or text == "--START DEREGISTRATION--":
                break
            relevant_texts.append(text)
        relevant_texts.reverse()

        for text in relevant_texts:
            for course_name in course_names:
                if course_name.lower() in text.lower():
                    latest_course_name = course_name
                    break
            if latest_course_name:
                break
        return latest_course_name


    def register_exam(self):
            
        insert_position = max(len(self.answers) - 1, 0)  # Ensure it's not negative
        self.answers.insert(insert_position, "--START REGISTRATION--")
        # get matriculation from previous answers
        matriculation_number = self.find_matriculation_number(self.answers)
        while matriculation_number is None:
            matriculation_number = self.find_matriculation_number([self.chat("What is your Matriculation Number?")])

        # Check if student exists 
        student = self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        while student is None:
            print("Sorry, you are not registered as a student. Make sure you enter the correct Matriculation Number")
            matriculation_number = self.find_matriculation_number([self.chat("What is your Matriculation Number?")])
            student = self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        
        def get_avail_courses():
            avail_courses = self.db.get_many(f"""
            SELECT c.course_name
            FROM courses c
            LEFT JOIN registrations r ON c.course_id = r.course_id AND r.matriculation_number = {matriculation_number}
            LEFT JOIN exam_results er ON c.course_id = er.course_id AND er.matriculation_number = {matriculation_number}
            WHERE r.matriculation_number IS NULL AND er.matriculation_number IS NULL;
            """)
            print("You can only register for the following courses:")
            for i, course_tuple in enumerate(avail_courses, start=1):
                course_name = course_tuple[0]  # Extract the course name from the tuple
                print(f"{i}. {course_name}")


        course_name = self.extract_course_name(self.answers)

        if course_name is None:
            self.chat("Please enter the name of the course you want to register for: ")
            course_name = self.extract_course_name(self.answers)
       

        course = self.db.get_one('SELECT course_id FROM courses WHERE course_name = ?', (course_name,))
                
        while course is None:
            get_avail_courses()
            self.chat("Incorrect course name, please provide the correct one: ")
            course_name = self.extract_course_name(self.answers)
            course = self.db.get_one('SELECT course_id FROM courses WHERE course_name = ?', (course_name,))
        
        course_id = course[0]
        answer = self.chat(f"Let me summarize once again: you want to register for the exam {course_name}?")
        processed_answer = self.identify_yes_no_answer(answer)

        while processed_answer == 'unknown_intent':
            answer = self.chat("Sorry, I didn't understand. Please answer with yes or no.")
            processed_answer = self.identify_yes_no_answer(answer)

        if processed_answer == 'yes':
            # Check if already registered for the course
            registration = self.db.get_one('SELECT * FROM registrations WHERE matriculation_number = ? AND course_id = ?', (matriculation_number, course_id))
            result = self.db.get_many(f'SELECT grade FROM exam_results WHERE matriculation_number={matriculation_number} AND course_id={course_id}')
            if registration:
                print(f"Student with matriculation number {matriculation_number} is already registered for course {course_name}.")  

            # check if already graded for the course
            elif result:
                print(f"You are already graded for the Course {course_name}.")
                
            # Perform registration
            else:
                self.db.execute_mutation('INSERT INTO registrations (matriculation_number, course_id) VALUES (?, ?)', (matriculation_number, course_id))
                print(f"Student with matriculation number {matriculation_number} registered for course {course_name}.")
                
        elif processed_answer == 'no':
            return           


    def deregister_exam(self):

        insert_position = max(len(self.answers) - 1, 0)  # Ensure it's not negative
        self.answers.insert(insert_position, "--START DEREGISTRATION--")
        # get matriculation from previous answers
        matriculation_number = self.find_matriculation_number(self.answers)
        while matriculation_number is None:
            matriculation_number = self.find_matriculation_number([self.chat("What is your Matriculation Number?")])

        # Check if student exists
        student = self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        while student is None:
            print("Sorry, you are not registered as a student. Make sure you enter the correct Matriculation Number")
            matriculation_number = self.find_matriculation_number([self.chat("What is your Matriculation Number?")])
            student = self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        
        def get_avail_courses():
            avail_courses = self.db.get_many(f"""
            SELECT c.course_name
            FROM registrations r
            INNER JOIN courses c ON r.course_id = c.course_id AND r.matriculation_number = {matriculation_number}
            """)
            print("You can only deregister for the following courses:")
            for i, course_tuple in enumerate(avail_courses, start=1):
                course_name = course_tuple[0]  # Extract the course name from the tuple
                print(f"{i}. {course_name}")

        course_name = self.extract_course_name(self.answers)

        if course_name is None:
            self.chat("Please enter the name of the course you want to deregister for: ") 
            course_name = self.extract_course_name(self.answers)
        
        course = self.db.get_one('SELECT course_id FROM courses WHERE course_name = ?', (course_name,))

        while course is None:
            get_avail_courses()
            self.chat("Incorrect course name, please provide the correct one: ")
            course_name = self.extract_course_name(self.answers)
            course = self.db.get_one('SELECT course_id FROM courses WHERE course_name = ?', (course_name,))
            
        course_id = course[0]
        answer = self.chat(f"Let me summarize once again: you want to deregister from the exam {course_name}?")
        processed_answer = self.identify_yes_no_answer(answer)

        while processed_answer == 'unknown_intent':
            answer = self.chat("Sorry, I didn't understand. Please answer with yes or no.")
            processed_answer = self.identify_yes_no_answer(answer)

        if processed_answer == 'yes':
            # Check if actually registered
            registration = self.db.get_one('SELECT * FROM registrations WHERE matriculation_number = ? AND course_id = ?', (matriculation_number, course_id))
            if not registration:
                print(f"Student with matriculation number {matriculation_number} is not registered for course {course_name}.")

            # Perform deregistration
            else:
                self.db.execute_mutation('DELETE FROM registrations WHERE matriculation_number = ? AND course_id = ?', (matriculation_number, course_id))
                print(f"Student with matriculation number {matriculation_number} deregistered from course {course_name}.")
                   
        elif processed_answer == 'no':
            return  # Break the loop if the user says no, and ask start from beginning


    def extract_info(self, tree, label):
        info_list = []
        for subtree in tree.subtrees():
            if subtree.label() == label:
                info_list.append(" ".join(word for word, tag in subtree.leaves()))
        return info_list


    def get_post_code(self,user_input):
        postcode_pattern = re.compile(r'\b\d{5}\b')
        result = re.search(postcode_pattern, user_input)
        first_match = result.group(0) if result else None
        return first_match


    def change_address(self):
        matriculation_number = self.find_matriculation_number(self.answers)
        while matriculation_number is None:
            self.chat("Please enter your matriculation number: ")
            matriculation_number = self.find_matriculation_number(self.answers)

        #Check if student exists
        student=  self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        if student is None:
            print("Sorry, you are not registered as a student.")
        else:

            #Ask for new address
            def validate_address_input(new_address, is_final_answer = False):
                tokens = word_tokenize(new_address)
                tagged = nltk.pos_tag(tokens)
                # Grammar for City and Postal Number
                city_grammar = r"""
                    CITY: {<CD>?<NNP|NN><CD>?}
                """
                city_cp = nltk.RegexpParser(city_grammar)
                city_result = city_cp.parse(tagged)
                city_combinations = self.extract_info(city_result, 'CITY')
                temp_city_filtered_combinations = [item for item in city_combinations if self.get_post_code(item) is not None]
                if len(temp_city_filtered_combinations) > 0:
                    self.city_filtered_combinations = temp_city_filtered_combinations
                street_grammar = r"""
                STREET: {<DT|NNP>?<NNP|NN>?<NNP><CD>}
                """
                street_cp = nltk.RegexpParser(street_grammar)
                street_result = street_cp.parse(tagged)
                street_pattern = r"((Ober|Unter den|An |Im |Platz |Berg |Am |Alt\-).+|(?:([A-Z][a-zäüö-]+){1,2})).([Cc]haussee|[Aa]llee|[sS]tr(\.|(a(ss|ß)e))|[Rr]ing|berg|gasse|grund|hörn| Nord|graben|[mM]arkt|[Uu]fer|[Ss]tieg|[Ll]inden|[Dd]amm|[pP]latz|brücke|Steinbüchel|Burg|stiege|[Ww]eg|rain|park|[Ww]eide|[Hh][oö]f|pfad|garten|bogen).+?(\d{1,4})([a-zäöüß]+)?(\-?\d{1,4}[a-zäöüß]?)?"
                street_combination = self.extract_info(street_result, 'STREET')
                temp_street_filtered_combinations = [item for item in street_combination if re.match(street_pattern, item)]
                if len(temp_street_filtered_combinations) > 0:
                    self.street_filtered_combinations = temp_street_filtered_combinations
                    
                


                if is_final_answer:
                    first_while_counter = 0
                    while len(self.street_filtered_combinations) == 0 and len(self.city_filtered_combinations) == 0:
                        prompt = "Please make sure you have entered your street name and number as well as your city and postal code correctly."
                        if first_while_counter == 0:
                            prompt = "Can you give me your addresss?"

                        new_street_input = self.chat(prompt)
                        tokens = word_tokenize(new_street_input)
                        tagged = nltk.pos_tag(tokens)
                        street_result = street_cp.parse(tagged)
                        street_combination = self.extract_info(street_result, 'STREET')
                        temp_street_filtered_combinations = [item for item in street_combination if re.match(street_pattern, item)]
                        if len(temp_street_filtered_combinations) > 0:
                            self.street_filtered_combinations = temp_street_filtered_combinations
                        first_while_counter += 1
                        
                        city_result = city_cp.parse(tagged)
                        city_combinations = self.extract_info(city_result, 'CITY')
                        temp_city_filtered_combinations = [item for item in city_combinations if self.get_post_code(item) is not None]
                        if len(temp_city_filtered_combinations) > 0:
                            self.city_filtered_combinations = temp_city_filtered_combinations



                    while len(self.street_filtered_combinations) == 0:
                        new_street_input = self.chat("Please make sure you have entered your street name and number correctly.")
                        tokens = word_tokenize(new_street_input)
                        tagged = nltk.pos_tag(tokens)
                        street_result = street_cp.parse(tagged)
                        street_combination = self.extract_info(street_result, 'STREET')
                        temp_street_filtered_combinations = [item for item in street_combination if re.match(street_pattern, item)]
                        if len(temp_street_filtered_combinations) > 0:
                            self.street_filtered_combinations = temp_street_filtered_combinations

                    
                    while len(self.city_filtered_combinations) == 0:
                        new_city_input = self.chat("Please make sure you have entered your city and postal code correctly.")
                        tokens = word_tokenize(new_city_input)
                        tagged = nltk.pos_tag(tokens)
                        city_result = city_cp.parse(tagged)
                        city_combinations = self.extract_info(city_result, 'CITY')
                        temp_city_filtered_combinations = [item for item in city_combinations if self.get_post_code(item) is not None]
                        if len(temp_city_filtered_combinations) > 0:
                            self.city_filtered_combinations = temp_city_filtered_combinations

                    new_address = self.street_filtered_combinations[0] + ", " + self.city_filtered_combinations[0]
                    return new_address
                

            for index, answer in enumerate(reversed(self.answers)):
                final_address = validate_address_input(answer, index == len(self.answers) - 1)
                if final_address is not None:
                    break
            final_answer = self.chat("Let me summarize once again: You have moved out and your new address is " + final_address + "? ")
            while final_answer == 'no':
                self.street_filtered_combinations = []
                self.city_filtered_combinations = []
                new_address = self.chat("Please enter your new address ")
                final_address = validate_address_input(new_address, True)
                final_answer = self.chat("Let me summarize once again: You have moved out and your new address is " + final_address + "? ")
            
            self.db.execute_mutation('UPDATE students SET address=? WHERE matriculation_number=?', (final_address, matriculation_number))
            print("Your address has been updated.")


    def suggest_course(self):
        # Request the student's matriculation number
        self.chat("Please enter your matriculation number: ")
        matriculation_number = self.find_matriculation_number(self.answers)
        while matriculation_number is None:
            self.chat("Please enter your matriculation number: ")
            matriculation_number = self.find_matriculation_number(self.answers)
            
        student = self.db.get_one('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
        if student is None:
            self.chat("Sorry, you are not registered as a student.")
            return

        # Check if the student has registered for any courses
        registered_courses_query = f"SELECT course_id FROM registrations WHERE matriculation_number = '{matriculation_number}'"
        registered_courses = self.db.get_many(registered_courses_query)
        if not registered_courses:
            self.chat("No registered courses found for matriculation number: " + matriculation_number)
            return

        # Extract just the course IDs from the query results
        registered_course_ids = [course[0] for course in registered_courses]

        print(registered_course_ids)

        # Course recommendation rules
        course_recommendations = [
            ([102, 108], 106),  # Advanced Mathematics + Software Engineering -> Artificial Intelligence
            ([105, 104], 111),  # Database Systems + Data Structures and Algorithms -> Computer Networks
            ([101, 103], 109),  # Introduction to Computer Science + Physics for Engineers -> Web Development
            ([104, 106], 107),  # Data Structures and Algorithms + Artificial Intelligence -> Machine Learning
            ([109, 105], 108),  # Web Development + Database Systems -> Software Engineering
            ([110, 104], 108),  # Operating Systems + Data Structures and Algorithms -> Software Engineering
            ([105, 102], 107),  # Database Systems + Advanced Mathematics -> Machine Learning
            ([106, 103], 111),  # Artificial Intelligence + Physics for Engineers -> Computer Networks
            ([107, 109], 105),  # Machine Learning + Web Development -> Database Systems
            ([111, 110], 102),  # Computer Networks + Operating Systems -> Advanced Mathematics
            ([101, 108], 104),  # Introduction to Computer Science + Software Engineering -> Data Structures and Algorithms
            ([103, 102], 106),  # Physics for Engineers + Advanced Mathematics -> Artificial Intelligence
            ([106, 105], 104),  # Artificial Intelligence + Database Systems -> Data Structures and Algorithms
        ]

        # Iterate through the course recommendations and check if the student meets the prerequisites
        for prerequisites, recommendation in course_recommendations:
            if all(prerequisite in registered_course_ids for prerequisite in prerequisites):
                recommended_course_name = self.db.get_one("SELECT course_name FROM courses WHERE course_id=?", (recommendation,))
                self.chat(f"Based on your previously taken courses, I suggest you take the course: {recommended_course_name[0]}.")
                return

        # If no recommendation was found
        self.chat("Based on your previously taken courses, there are no specific recommendations. Consider another university.")


chatbot = ChatBot()
chatbot.start_conversation()